# Sprint 13 - Proyecto

#### Cargar las librerías

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    f1_score,
    mean_squared_error
)

# Utilities
from sklearn.utils import shuffle


In [2]:
#Cargar los archivos

gold_recovery_train = pd.read_csv('data/gold_recovery_train.csv')
gold_recovery_test = pd.read_csv('data/gold_recovery_test.csv')
gold_recovery_full = pd.read_csv('data/gold_recovery_full.csv')

In [34]:
#Análisis exploratorio de datos para los tres conjuntos:

# Para cada dataset
print("Dimensiones del datasets:")
print(f"Train: {gold_recovery_train.shape}")
print(f"Test: {gold_recovery_test.shape}") 
print(f"Full: {gold_recovery_full.shape}")
print()
print("Valores ausentes de los datasets:")
print(f"Train - Total valores ausentes: {gold_recovery_train.isnull().sum().sum()}, es decir, el {(gold_recovery_train.isnull().sum().sum()) / gold_recovery_train.size * 100} está ausente")
print(f"Test - Total valores ausentes: {gold_recovery_test.isnull().sum().sum()}, es decir, el {(gold_recovery_test.isnull().sum().sum()) / gold_recovery_test.size * 100} está ausente")
print(f"Full - Total valores ausentes: {gold_recovery_full.isnull().sum().sum()}, es decir, el {(gold_recovery_full.isnull().sum().sum()) / gold_recovery_full.size * 100} está ausente")

Dimensiones del datasets:
Train: (16860, 87)
Test: (5856, 53)
Full: (22716, 87)

Valores ausentes de los datasets:
Train - Total valores ausentes: 30320, es decir, el 2.0670566258982013 está ausente
Test - Total valores ausentes: 2360, es decir, el 0.7603876688318383 está ausente
Full - Total valores ausentes: 36587, es decir, el 1.851295253940207 está ausente


In [ ]:
# Análisis para el conjunto de entrenamiento

print('== Información general sobre el conjunto de entrenamiento ==')
print(gold_recovery_train.info())
print()
print()
print('== Primeras filas del conjunto de entrenamiento ==')
print(gold_recovery_train.head(10))
print()
print('== Primeras filas del conjunto de entrenamiento ==')
print(gold_recovery_train.head(10))


== Información general sobre el conjunto de entrenamiento ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.ta

In [26]:
print(gold_recovery_train.size)

1466820
